In [15]:
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

# Human Prompt Template

In [16]:
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages = [
        HumanMessagePromptTemplate.from_template(
            "{content}"
        )
    ]
)

In [17]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain


chat = ChatOpenAI(
    model='gpt-3.5-turbo',
)



# Create the chain with ChatModel

In [18]:
chain = prompt | chat

# Application: Human Prompt Template

In [22]:
def chat_bot(content):
    # content = input("You: ")
    
    result = chain.invoke({
        "content": "hello"
    })

    print("AI: ", result.content)

In [23]:
chat_bot("hello")

AI:  Hello! How can I assist you today?


In [24]:
chat_bot("how are you?")

AI:  Hello! How can I assist you today?


# Introduction to Memory

#### For Completion Type Models

- ConversationTokenBufferMemory
- CombinedMemory
- ConverstaionBufferWindowMemory

#### For Chat Models

- ConversationBufferMemory

The Memroy adds 'HumanMessage' and 'AiMessage' to the list.

**Memory doesn't add the history and push them to LLM**

In [25]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

In [38]:
memory = ConversationBufferMemory(
    memory_key="messages",
    return_messages=True
)

In [39]:
prompt_with_memory = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template(
            "{content}"
        )
    ]
)

In [40]:
chain_with_memory = LLMChain(
    prompt= prompt_with_memory,
    memory= memory,
    llm= chat
)

In [43]:
def chat_bot_with_memory(content):
    result = chain_with_memory.invoke({
        "content": content,
        "messages": []
    })

    print("AI: ", result['text'])


In [44]:
chat_bot_with_memory("what is 1 + 1?")
chat_bot_with_memory("and 3 more?")

AI:  1 + 1 = 2
AI:  1 + 1 + 3 = 5


# Langchain with storage

In [46]:
from langchain.memory import FileChatMessageHistory

llm_with_history = ChatOpenAI(
    model='gpt-3.5-turbo',
)

In [47]:
memory_with_history = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [48]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template(
            "{content}"
        )
    ]
)

In [51]:
chain_with_history = LLMChain(
    prompt= prompt,
    memory= memory_with_history,
    llm= llm_with_history
)

In [54]:
def chat_bot_with_history(content):
    result = chain_with_history.invoke({
        "content": content,
        "messages": []
    })

    print("AI: ", result['text'])

In [55]:
chat_bot_with_history("what is 1 + 1?")
chat_bot_with_history("add 3 more?")

AI:  1 + 1 = 2
AI:  2 + 3 = 5


### Optimised Memory using ConverstaionSummaryMemory

In [63]:
from langchain.memory import ConversationSummaryMemory


optimised_memory = ConversationSummaryMemory(
    memory_key="messages",
    return_messages=True,
    llm=chat
)

optimised_prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template(
            "{content}"
        )
    ]
)


optimised_chain = LLMChain(
    prompt=optimised_prompt,
    memory=optimised_memory,
    llm=chat,
    verbose=True
)

In [59]:
def chat_bot_optimised(content):
    result = optimised_chain.invoke({
        "content": content,
        "messages": []
    })

    print("AI: ", result['text'])

In [64]:
chat_bot_optimised("what is 1 + 1?")
chat_bot_optimised("add 3 more?")
chat_bot_optimised("again 5 more?")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
Human: what is 1 + 1?

> Finished chain.
AI:  1 + 1 equals 2.


> Entering new LLMChain chain...
Prompt after formatting:
System: The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then asks what is 1 + 1, to which the AI responds that 1 + 1 equals 2.
Human: add 3 more?

> Finished chain.
AI:  1 + 1 + 3 equals 5.


> Entering new LLMChain chain...
Prompt after formatting:
System: The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then asks what is 1 + 1, to which the AI responds that 1 + 1 equals 2. When asked to add 3 more, the AI calculates that 1 + 1 + 3 equals 5.
Human: again 5 more?

> Finished chain.
AI:  Adding 5 more to the previous result of 1 